In [1]:
import numpy as np
from keras.datasets import cifar10
from keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Bidirectional, Dropout, Conv2D, MaxPooling2D, GlobalMaxPool2D, Input, Embedding, SimpleRNN, BatchNormalization
from keras import optimizers
from keras.regularizers import l2
from keras.models import Sequential, Model
from math import ceil


(X_train, y_train), (X_test, y_test) = cifar10.load_data()

def save_bottleneck_features():

    model = VGG16(include_top=False, weights='imagenet')

    generator = ImageDataGenerator(rescale=1. / 255)
    train_datagen = generator.flow(X_train, y_train, batch_size=32, shuffle=False)
    bottleneck_features_train = model.predict(train_datagen, steps=ceil(len(X_train) / 32))

    test_datagen = generator.flow(X_test, y_test, batch_size=32, shuffle=False)
    bottleneck_features_validation = model.predict(test_datagen, steps=ceil(len(X_test) / 32))

    np.save('bottleneck_features_train.npy', bottleneck_features_train)
    np.save('bottleneck_features_validation.npy', bottleneck_features_validation)

def train_top_model():
    train_data = np.load(open('bottleneck_features_train.npy', 'rb'))
    test_data = np.load(open('bottleneck_features_validation.npy', 'rb'))

    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax')) 
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    model.fit(train_data, y_train, epochs=50, batch_size=32, validation_data=(test_data, y_test))

save_bottleneck_features()
train_top_model()

2024-02-18 17:24:05.375131: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-18 17:24:05.375175: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-18 17:24:05.376197: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-18 17:24:05.382144: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-18 17:24:06.462993: W tensorflow/compiler/tf2

: 